In [2]:
from DataFetcher import DataFetcher
from USTs import USTs
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime

In [3]:
auctions = DataFetcher().fetch_auction_data()
prices = DataFetcher().fetch_historical_UST_data(date=datetime(year=2025, month=6, day=17))

In [4]:
auctions

,cusip,auction_date,issue_date,security_term,maturity_date,avg_med_yield,currently_outstanding,original_security_term
0,91282CNJ6,2025-06-26,2025-06-30,7-Year,2032-06-30,null,null,7-Year
1,91282CMX6,2025-06-25,2025-06-27,1-Year 10-Month,2027-04-30,null,60044000000.000000,2-Year
2,91282CNK3,2025-06-25,2025-06-30,5-Year,2030-06-30,null,null,5-Year
3,91282CNL1,2025-06-24,2025-06-30,2-Year,2027-06-30,null,null,2-Year
4,912797MG9,2025-06-24,2025-06-26,6-Week,2025-08-07,null,207829000000.000000,52-Week
...,...,...,...,...,...,...,...,...
75,9127935W2,1980-01-02,1980-01-08,52-Week,1981-01-02,null,null,52-Week
76,9127933Y0,1979-12-28,1980-01-03,13-Week,1980-04-03,null,null,null
77,9127934U7,1979-12-28,1980-01-03,26-Week,1980-07-03,null,null,26-Week
78,912810CK2,1979-11-01,1979-11-15,30-Year,2009-11-15,10.440000,null,30-Year


In [5]:
ust_set = USTs(auction_data=auctions, price_data=prices).get_current_UST_set()

Merged auction and price data successfully
No missing or excess data
All CUSIPs are identical between DataFrames


In [6]:
from datetime import timedelta
ust_set['days'] = ust_set['Maturity date'] - ust_set['issue_date']

In [10]:
ust_set[ust_set['security_term'] == '10-Year'].head(30)

,Cusip,Security type,Rate,Maturity date,Buy,Sell,End of day,security_term,issue_date,days
57,912828K74,Note,2.000,2025-08-15,0.000000,99.59375,99.59375,10-Year,2015-08-17,3651 days
70,912828M56,Note,2.250,2025-11-15,0.000000,99.12500,99.12500,10-Year,2015-11-16,3652 days
83,912828P46,Note,1.625,2026-02-15,98.328125,98.28125,98.28125,10-Year,2016-02-16,3652 days
96,912828R36,Note,1.625,2026-05-15,97.750000,97.71875,97.71875,10-Year,2016-05-16,3651 days
109,9128282A7,Note,1.500,2026-08-15,97.078125,97.06250,97.06250,10-Year,2016-08-15,3652 days
122,912828U24,Note,2.000,2026-11-15,97.203125,97.18750,97.18750,10-Year,2016-11-15,3652 days
134,912828V98,Note,2.250,2027-02-15,97.203125,97.18750,97.18750,10-Year,2017-02-15,3652 days
147,912828X88,Note,2.375,2027-05-15,97.093750,97.06250,97.09375,10-Year,2017-05-15,3652 days
158,9128282R0,Note,2.250,2027-08-15,96.515625,96.50000,96.50000,10-Year,2017-08-15,3652 days
168,9128283F5,Note,2.250,2027-11-15,96.171875,96.15625,96.18750,10-Year,2017-11-15,3652 days
